In [1]:
# How to Run Django in Jupyter Notebook
# https://www.youtube.com/watch?v=TN5zIA--cz4&ab_channel=TechSunami

In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.db import connection
from django.db import reset_queries
from django.db import models
from django_orm_app.models import Author, Blog
from faker import Faker 
import random

In [2]:
def define_model(model):
    print(f"----------------- {model.__name__}-------------------")
    for field in model._meta.fields:
        print(f"{field.name} --> {field.get_internal_type()}")
    print('\033[1m' + f"Table name in database is {model._meta.db_table}" + '\033[0m')

In [3]:
define_model(Author)
define_model(Blog)

----------------- Author-------------------
id --> BigAutoField
name --> CharField
email --> CharField
age --> IntegerField
creation_date --> DateTimeField
updation_date --> DateTimeField
Table name in database is author_model
----------------- Blog-------------------
id --> BigAutoField
name --> CharField
author --> ForeignKey
tagline --> TextField
creation_date --> DateTimeField
updation_date --> DateTimeField
Table name in database is blog_model


### Generating Random Data

In [5]:
# fake = Faker()
# range_of_author_id = 100

In [6]:
# lst_author = []
# for _ in range(range_of_author_id):
#     lst_author.append(
#         Author(
#             name=fake.name(), 
#             age=random.randint(1,100)
#         )
#     )

In [7]:
# objs_author = Author.objects.bulk_create(lst_author)

In [8]:
# lst_blog = []
# for _ in range(10000):
#     lst_blog.append(
#         Blog(
#             name=fake.sentence(), 
#             tagline= fake.paragraph(), 
#             author_id=random.randint(1, 100)
#         )
#     )

In [9]:
# objs_blog = Blog.objects.bulk_create(lst_blog)

## Table of Content
1. [Model Creation Type](#1)
2. [Understand Query Evaluation](#2)
    1. [Queryset are Lazy](#2.a) 
    2. [How Queryset held data in Memory](#2.b)
3. [Method that don't return Queryset](#3)
    1. [get](#3.a) 
    2. [get_or_create](#3.b) 
    3. [update_or_create ](#3.c) 
    4. [bulk_create](#3.d) 
    5. [bulk_update](#3.e) 
    6. [count](#3.f) 
    7. [latest](#3.g) 
    8. [earliest](#3.h) 
    9. [last](#3.i) 
    10. [first](#3.j) 
    11. [exists](#3.k) 
    12. [contains](#3.l)
    13. [delete](#3.m)
    14. [update](#3.n)
    15. [aggregate](#3.o)

In [10]:
def get_query():
    return connection.queries

reset_queries()

<a id="1"></a>
## Model Creation Type


In [4]:
author_instance = Author(
    name = "Lakshit", 
    email="lakshit@gmail.com"
)
author_instance.save()

In [5]:
blog_instance = Blog.objects.create(
    name="blog no 1", 
    tagline="All the latest bloggggg news.",
    author=author_instance
)

In [6]:
blog_dict = {
    "name":"Blog number 2", 
    "tagline":"This is best blog everrrrrrrrr......",
    "author":author_instance
}
blog_instance_2 = Blog.objects.create(**blog_dict)

In [7]:
get_query()

NameError: name 'get_query' is not defined

In [10]:
reset_queries()

A <b>QuerySet</b> represents a collection of objects from your database. It can have zero, one or many filters. Filters narrow down the query results based on the given parameters. In SQL terms, a QuerySet equates to a SELECT statement, and a filter is a limiting clause such as WHERE or LIMIT .

<a id="2"></a>
## Understand Query Evaluation

<a id="2.a"></a>
### Queryset are Lazy
<pre>QuerySets are lazy – the act of creating a QuerySet doesn’t involve any database activity. You can stack filters together all day long, and Django won’t actually run the query until the QuerySet is evaluated. Take a look  at below example:</pre>

Django Documentation - https://docs.djangoproject.com/en/4.2/topics/db/queries/#querysets-are-lazy

In [11]:
q = Blog.objects.filter(id = 1)

In [12]:
q

<QuerySet [<Blog: blog no 1>]>

In [13]:
get_query()

[{'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" WHERE "blog_model"."id" = 1 LIMIT 21',
  'time': '0.001'}]

In [47]:
reset_queries()

<a id="2.b"></a>
### How Queryset held data in Memory

Django Documentation - https://docs.djangoproject.com/en/4.2/topics/db/queries/#caching-and-querysets

In [30]:
q = Blog.objects.all()

In [41]:
q[0]

<Blog: lakshit>

In [42]:
get_query()

[{'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" LIMIT 1',
  'time': '0.000'},
 {'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" LIMIT 1',
  'time': '0.000'},
 {'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" LIMIT 1',
  'time': '0.000'},
 {'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model"',
  'time': '0.001'}]

In [37]:
print([p.name for p in q])

['lakshit', 'Blog number 2', 'blog no 1', 'Blog number 2']


In [21]:
Blog.objects.filter(id = 1).update(name = "lakshit")

1

In [19]:
q[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x10a8c8ac0>,
 'id': 1,
 'name': 'blog no 1',
 'author_id': 1,
 'tagline': 'All the latest bloggggg news.',
 'creation_date': datetime.datetime(2023, 7, 4, 20, 34, 40, 924446, tzinfo=datetime.timezone.utc),
 'updation_date': datetime.datetime(2023, 7, 4, 20, 34, 40, 924496, tzinfo=datetime.timezone.utc)}

In [25]:
q[0].name

'lakshit'

<a id="3"></a>
## Method that don't return Queryset
Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#methods-that-do-not-return-querysets

<a id="3.a"></a>
### get

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#get

In [43]:
obj = Blog.objects.get(id = 1)

In [44]:
try:
    obj = Blog.objects.get(id = 100)
except Blog.DoesNotExist:
    print("In Does Not exist Block")
except Blog.MultipleObjectsReturned:
    print("In Multi Object Block")

In Does Not exist Block


<a id="3.b"></a>
### get_or_create

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#get-or-create

In [48]:
#Traditional Way
try:
    obj = Blog.objects.get(id = 100)
except Blog.DoesNotExist:
    blog_dict = {
        "name":"Blog number 2", 
        "tagline":"This is best blog everrrrrrrrr......"
    }
    obj = Blog.objects.create(**blog_dict)

In [51]:
get_query()

[{'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" WHERE "blog_model"."id" = 100 LIMIT 21',
  'time': '0.001'},
 {'sql': 'INSERT INTO "blog_model" ("name", "author_id", "tagline", "creation_date", "updation_date") VALUES (\'Blog number 2\', NULL, \'This is best blog everrrrrrrrr......\', \'2023-07-04 20:40:44.004965\', \'2023-07-04 20:40:44.005007\') RETURNING "blog_model"."id"',
  'time': '0.001'},
 {'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" WHERE "blog_model"."id" = 100 LIMIT 21',
  'time': '0.000'},
 {'sql': 'BEGIN', 'time': '0.000'},
 {'sql': 'INSERT INTO "blog_model" ("id", "name", "author_id", "tagline", "creation_date", "updation_date") VALUES (100, \'Blog number 2\', NULL, \'This is best blog everrrrrrrrr.....

In [50]:
obj, created = Blog.objects.get_or_create(
    id=100,
    defaults={
        "name":"Blog number 2", 
        "tagline":"This is best blog everrrrrrrrr......"
    },
)

<a id="3.c"></a>
### update_or_create 
Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#update-or-create

In [55]:
search_field = {"name": "Lakshit"}
update_field = {"name": "Techsunami"}
try:
    obj = Author.objects.get(**search_field)
    for key, value in update_field.items():
        setattr(obj, key, value)
    obj.save()
except Author.DoesNotExist:
    new_values = search_field
    new_values.update(update_field)
    obj = Author(**new_values)
    obj.save()

In [62]:
get_query()

[{'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" WHERE "blog_model"."id" = 100 LIMIT 21',
  'time': '0.001'},
 {'sql': 'INSERT INTO "blog_model" ("name", "author_id", "tagline", "creation_date", "updation_date") VALUES (\'Blog number 2\', NULL, \'This is best blog everrrrrrrrr......\', \'2023-07-04 20:40:44.004965\', \'2023-07-04 20:40:44.005007\') RETURNING "blog_model"."id"',
  'time': '0.001'},
 {'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" WHERE "blog_model"."id" = 100 LIMIT 21',
  'time': '0.000'},
 {'sql': 'BEGIN', 'time': '0.000'},
 {'sql': 'INSERT INTO "blog_model" ("id", "name", "author_id", "tagline", "creation_date", "updation_date") VALUES (100, \'Blog number 2\', NULL, \'This is best blog everrrrrrrrr.....

In [56]:
obj

<Author: Techsunami>

In [59]:
obj, created = Author.objects.update_or_create(
    name="Lakshit",
    defaults={"name": "Techsunami"},
)

<a id="3.d"></a>
### bulk_create

<pre>
<b>Note</b>
 - The model’s save() method will not be called, and the pre_save and post_save signals will not be sent.
</pre>

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#bulk-create/

In [63]:
reset_queries()

In [64]:
get_query()

[]

In [65]:
objs = Blog.objects.bulk_create(
    [
        Blog(name="Blog number 3"),
        Blog(name="Blog number 4"),
        Blog(name="Blog number 5")
    ]
)

In [66]:
get_query()

[{'sql': 'BEGIN', 'time': '0.000'},
 {'sql': 'INSERT INTO "blog_model" ("name", "author_id", "tagline", "creation_date", "updation_date") VALUES (\'Blog number 3\', NULL, \'\', \'2023-07-04 20:44:37.288404\', \'2023-07-04 20:44:37.288447\'), (\'Blog number 4\', NULL, \'\', \'2023-07-04 20:44:37.288471\', \'2023-07-04 20:44:37.288485\'), (\'Blog number 5\', NULL, \'\', \'2023-07-04 20:44:37.288506\', \'2023-07-04 20:44:37.288520\') RETURNING "blog_model"."id"',
  'time': '0.001'},
 {'sql': 'COMMIT', 'time': '0.001'}]

<a id="3.e"></a>
### bulk_update

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#bulk-update

In [67]:
objs[0].name = "SOmething 1"
objs[1].name = "something 2"
objs[2].name = "something 3"

Blog.objects.bulk_update(objs, ["name"])

3

In [68]:
objs

[<Blog: SOmething 1>, <Blog: something 2>, <Blog: something 3>]

In [69]:
get_query()

[{'sql': 'BEGIN', 'time': '0.000'},
 {'sql': 'INSERT INTO "blog_model" ("name", "author_id", "tagline", "creation_date", "updation_date") VALUES (\'Blog number 3\', NULL, \'\', \'2023-07-04 20:44:37.288404\', \'2023-07-04 20:44:37.288447\'), (\'Blog number 4\', NULL, \'\', \'2023-07-04 20:44:37.288471\', \'2023-07-04 20:44:37.288485\'), (\'Blog number 5\', NULL, \'\', \'2023-07-04 20:44:37.288506\', \'2023-07-04 20:44:37.288520\') RETURNING "blog_model"."id"',
  'time': '0.001'},
 {'sql': 'COMMIT', 'time': '0.001'},
 {'sql': 'BEGIN', 'time': '0.000'},
 {'sql': 'UPDATE "blog_model" SET "name" = CASE WHEN ("blog_model"."id" = 101) THEN \'SOmething 1\' WHEN ("blog_model"."id" = 102) THEN \'something 2\' WHEN ("blog_model"."id" = 103) THEN \'something 3\' ELSE NULL END WHERE "blog_model"."id" IN (101, 102, 103)',
  'time': '0.001'},
 {'sql': 'COMMIT', 'time': '0.001'}]

<a id="3.f"></a>
### count

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#count

In [70]:
objs = Blog.objects.all()

In [71]:
# Not optimized
len(objs)
print(f"Query made is {get_query()[-1]}")

Query made is {'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model"', 'time': '0.001'}


In [72]:
# optimized
Blog.objects.count()
print(f"Query made is {get_query()[-1]}")

Query made is {'sql': 'SELECT COUNT(*) AS "__count" FROM "blog_model"', 'time': '0.001'}


<a id="3.g"></a>
### latest

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#latest

In [73]:
Blog.objects.latest("id", "creation_date")

<Blog: something 3>

In [76]:
get_query()[-1]

{'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" ORDER BY "blog_model"."id" DESC, "blog_model"."creation_date" DESC LIMIT 1',
 'time': '0.001'}

<a id="3.h"></a>
### earliest

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#earliest

In [77]:
Blog.objects.earliest("id")
print(f"Query made is {get_query()[-1]}")

Query made is {'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" ORDER BY "blog_model"."id" ASC LIMIT 1', 'time': '0.001'}


<a id="3.i"></a>
### last

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#last

In [78]:
Blog.objects.last()
print(f"Query made is {get_query()[-1]}")

Query made is {'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" ORDER BY "blog_model"."id" DESC LIMIT 1', 'time': '0.001'}


<a id="3.j"></a>
### first

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#first

In [79]:
Blog.objects.first()
print(f"Query made is {get_query()[-1]}")

Query made is {'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" ORDER BY "blog_model"."id" ASC LIMIT 1', 'time': '0.001'}


<a id="3.k"></a>
### exists

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#exists

In [95]:
objs = Blog.objects.filter(name="SOmething 1")

In [93]:
# Not optimized
if objs:
    print(f"Query made is {get_query()[-1]}")

Query made is {'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" WHERE "blog_model"."name" = \'SOmething 1\'', 'time': '0.001'}


In [96]:
# optimized
if objs.exists():
    print(f"Query made is {get_query()[-1]}")

Query made is {'sql': 'SELECT 1 AS "a" FROM "blog_model" WHERE "blog_model"."name" = \'SOmething 1\' LIMIT 1', 'time': '0.001'}


<a id="3.l"></a>
### contains

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#contains

In [101]:
objs = Blog.objects.filter(name="SOmething 1")

In [100]:
# Not optimized
if objs[0] in objs:
    print(f"Query made is {get_query()[-1]}")

Query made is {'sql': 'SELECT "blog_model"."id", "blog_model"."name", "blog_model"."author_id", "blog_model"."tagline", "blog_model"."creation_date", "blog_model"."updation_date" FROM "blog_model" WHERE "blog_model"."name" = \'SOmething 1\'', 'time': '0.000'}


In [102]:
#Optimized
if objs.contains(objs[0]):
    print(f"Query made is {get_query()[-1]}")

Query made is {'sql': 'SELECT 1 AS "a" FROM "blog_model" WHERE ("blog_model"."name" = \'SOmething 1\' AND "blog_model"."id" = 101) LIMIT 1', 'time': '0.000'}


<a id="3.m"></a>
### delete

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#delete

In [103]:
Blog.objects.filter(name="Blog number 3").delete()

(0, {})

<a id="3.n"></a>
### update
<pre>
<b>Note</b>
 - The model’s save() method will not be called, and the pre_save and post_save signals will not be sent.
</pre>

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#update

In [ ]:
# Not optimized
obj = Blog.objects.get(id=1)
obj.name = "Somenameeee"
obj.save()

print(f"Query made is {get_query()[-1]}")

In [ ]:
# Optimized because no need to bring in memory (more will learn in future video)
Blog.objects.filter(id=1).update(name="Donnnnn")
print(f"Query made is {get_query()[-1]}")

<a id="3.o"></a>
### aggregate
<pre> Operation happen in Database directly</pre>

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#aggregate

In [ ]:
reset_queries()

In [ ]:
get_query()

In [ ]:
from django.db.models import Count
Blog.objects.aggregate(number_of_entries = Count("author"))
print(f"Query made is {get_query()[-1]}")

In [ ]:
Blog.objects.aggregate(Count("author"))
print(f"Query made is {get_query()[-1]}")

There are more aggregation Function like Count - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#id6

<a id="3.p"></a>
### Reverse Foreign Key

Documentation - https://stackoverflow.com/questions/5437335/django-queryset-with-filtering-on-reverse-foreign-key

In [11]:
author_obj = Author.objects.get(id=5)

In [12]:
#If you have written related_name
author_obj.author_blog.all()

# #else
# author_obj.blog_set.all()

<QuerySet [<Blog: Low you without development buy necessary.>, <Blog: Consider as interview heavy those sport.>, <Blog: Significant will radio.>, <Blog: Everyone house city interesting lose.>, <Blog: Value unit lot past task effect.>, <Blog: Relationship glass ok defense message.>, <Blog: Purpose maintain under despite part.>, <Blog: Study little together.>, <Blog: Both staff light successful.>, <Blog: Newspaper affect time agency meet response to.>, <Blog: Our nor day boy everybody end movement forget.>, <Blog: Cell game pay must imagine five.>, <Blog: Success while his everything history letter strong.>, <Blog: Hot receive create.>, <Blog: Follow economic admit continue.>, <Blog: Expert star admit make likely focus open.>, <Blog: Campaign seem school water have.>, <Blog: Anything ball sport three.>, <Blog: Network moment he none office history just.>, <Blog: Break health from series represent increase each oil.>, '...(remaining elements truncated)...']>

In [ ]:
print(f"Query made is {get_query()[-1]}")

## Hey.... If You are liked the content Do subscribe to my Youtube Channel
https://www.youtube.com/channel/UCpuKtSx_Cu8_ikyhlKAWBYA?sub_confirmation=1

<b>Also You can connect with me on Linkedin</b> - https://www.linkedin.com/in/lakshit-ukani/